# Декораторы

![](images/ph2.jpg)

В Python функция - тоже объект.

Поэтому их можно передавать в другие функции:

In [1]:
from copy import deepcopy


def sort(l, compare_f = lambda x, y: x > y):
    l_2 = deepcopy(l)
    # да, сортировка не очень, но для примера пойдет
    for i in range(len(l_2)):
        for j in range(i + 1, len(l_2)):
            if compare_f(l_2[i], l_2[j]):
                l_2[i], l_2[j] = l_2[j], l_2[i]
    return l_2
    

print(sort([1, 2, 3, 2, -100, 100]))
print(sort([1, 2, 3, 2, -100, 100], compare_f=lambda x, y: x < y))

[-100, 1, 2, 2, 3, 100]
[100, 3, 2, 2, 1, -100]


### **Декоратор** - функция, которая в качестве параметра получает функцию и в качестве результата также возвращает функцию

In [2]:
# пример декоратора
def something(func):    
    def something_inside():
        print("Go in")
        func()
        print("Go out")
    return something_inside

In [3]:
def print_something():
    print("something")

print_something()

something


### Попробуемъ вызвать декоратор как обычную функцию

In [19]:
something(print_something)

<function __main__.something.<locals>.something_inside()>

### А теперь попробуем вызвать функцию, которую нам вернул something

In [5]:
something(print_something)()

Go in
something
Go out


### Но умные люди придумали более простой способ использования декораторов

In [7]:
@something
def print_something():
    print("something")


print_something()

Go in
something
Go out


### Иногда приходится использовать несколько декораторов для одной функции

In [9]:
def something(func):    
    def something_inside():
        print("Go in")
        func()
        print("Go out")
    return something_inside


def something2(func):    
    def something_inside():
        print("Go in x2")
        func()
        print("Go out x2")
    return something_inside


@something2
@something
def print_something():
    print("something")


print_something()

Go in x2
Go in
something
Go out
Go out x2


### Что будет, если возьмем 3 декоратора?

In [10]:
def something3(func):    
    def something_inside():
        print("Go in x3")
        func()
        print("Go out x3")
    return something_inside

In [11]:
@something3
@something2
@something
def print_something():
    print("something")


print_something()

Go in x3
Go in x2
Go in
something
Go out
Go out x2
Go out x3


![](images/ph1.jpeg)

### А если функция возвращает значения?

In [12]:
def something(func):    
    def something_inside():
        print("Go in")
        result = func()
        print("Go out")
        return result
    return something_inside

In [14]:
from random import randint


def return_random():
    return randint(0, 100)


b = something(return_random)()
print(f'b = {b}')

Go in
Go out
b = 69


In [15]:
@something
def return_random():
    return randint(0, 100)

b = return_random()
print(f'b = {b}')

Go in
Go out
b = 56


### А если функция принимает аргументы?

In [16]:
def something(func):    
    def something_inside(a, b):
        print(f"Go in; a = {a}, b = {b}")
        result = func(a, b)
        print(f"Go out; a = {a}, b = {b}")
        return result
    return something_inside

In [17]:
from random import randint


def return_random(a, b):
    return randint(a, b)


c = something(return_random)(5, 5)
print(f'c = {c}')

Go in; a = 5, b = 5
Go out; a = 5, b = 5
c = 5


In [18]:
@something
def return_random(a, b):
    return randint(a, b)

c = return_random(6, 6)
print(f'c = {c}')

Go in; a = 6, b = 6
Go out; a = 6, b = 6
c = 6


### А если не знаем количество аргументов? Или их наименование

In [22]:
def something(func):
    # можно и только args, и только kwargs. если не знаете, что это, вопите и спрашивайте :)
    def something_inside(*args, **kwargs):
        print(f"Go in; args = {args}, kwargs = {kwargs}")
        result = func(*args, **kwargs)
        print(f"Go out; args = {args}, kwargs = {kwargs}")
        return result
    return something_inside

In [23]:
from random import randint


def return_min_4(a, b, c, d):
    return min([a, b, c, d])

def return_min_5(a, b, c, d, e):
    return min([a, b, c, d, e])


c = something(return_min_4)(5, 5, c=8, d=9)
print(f'min4 = {c}', end='\n\n')

c = something(return_min_5)(5, 5, e=0, c=8, d=9)
print(f'min5 = {c}')

Go in; args = (5, 5), kwargs = {'c': 8, 'd': 9}
Go out; args = (5, 5), kwargs = {'c': 8, 'd': 9}
min4 = 5

Go in; args = (5, 5), kwargs = {'e': 0, 'c': 8, 'd': 9}
Go out; args = (5, 5), kwargs = {'e': 0, 'c': 8, 'd': 9}
min5 = 0


In [25]:
@something
def return_min_4(a, b, c, d):
    return min([a, b, c, d])

@something
def return_min_5(a, b, c, d, e):
    return min([a, b, c, d, e])

c = return_min_4(5, 5, c=8, d=9)
print(f'min4 = {c}', end='\n\n')

c = return_min_5(5, 5, e=0, c=8, d=9)
print(f'min5 = {c}')

Go in; args = (5, 5), kwargs = {'c': 8, 'd': 9}
Go out; args = (5, 5), kwargs = {'c': 8, 'd': 9}
min4 = 5

Go in; args = (5, 5), kwargs = {'e': 0, 'c': 8, 'd': 9}
Go out; args = (5, 5), kwargs = {'e': 0, 'c': 8, 'd': 9}
min5 = 0


### А могут ли декораторы сами принимать аргументы?

In [26]:
# спойлер: могут
import functools


def something(num=3):
    def something_inside(func):
        @functools.wraps(func)
        def something_inside2(*args, **kwargs):
            print(f"Go in x{num}; args = {args}, kwargs = {kwargs}")
            result = func(*args, **kwargs)
            print(f"Go out x{num}; args = {args}, kwargs = {kwargs}")
            return result
        return something_inside2
    return something_inside

In [27]:
def return_random(a, b):
    return randint(a, b)

something(num=5)

<function __main__.something.<locals>.something_inside(func)>

In [28]:
something(num=5)(return_random)

<function __main__.return_random(a, b)>

In [29]:
c = something(num=5)(return_random)(5, 100)
print(f"c = {c}")

Go in x5; args = (5, 100), kwargs = {}
Go out x5; args = (5, 100), kwargs = {}
c = 57


In [30]:
@something(5)
def return_random(a, b):
    return randint(a, b)

return_random(10, 20)

Go in x5; args = (10, 20), kwargs = {}
Go out x5; args = (10, 20), kwargs = {}


17

In [31]:
N = 10

@something(num=N)
def return_random(a, b):
    return randint(a, b)

return_random(10, 20)

Go in x10; args = (10, 20), kwargs = {}
Go out x10; args = (10, 20), kwargs = {}


20

In [ ]:
# перенос
запись
домашка
ссылка